In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [3]:
root_dir = Path().cwd().parent
calib_dir = root_dir / 'data/raw/final_project/calib'
interim_dir = root_dir / 'data/interim'
interim_dir.mkdir(parents=True, exist_ok=True)
undistorted_dir = interim_dir / 'undistorted/seq_01'
undistorted_dir.mkdir(parents=True, exist_ok=True)
undistort_dir = root_dir / 'data/raw/final_project/seq_01'

In [4]:
calib_folders = os.listdir(calib_dir)

for folder in calib_folders:
    images = os.listdir(calib_dir / folder / 'data')
    
    objpoints = []
    imgpoints = []
    
    im = cv2.imread(str(calib_dir / folder / 'data' / images[0]))
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    for v,h in zip([5,7,5], [7,11,15]):
        ret = True
        while ret:
            ret, corners = cv2.findChessboardCorners(im_gray, (v,h))
            #if not ret: ret, corners = cv2.findChessboardCornersSB(im_gray, (v,h))
            if not ret: break
            
            imgpoints.append(corners)
            objp = np.zeros((h*v,3), np.float32)
            objp[:,:2] = np.mgrid[0:v,0:h].T.reshape(-1,2)
            objpoints.append(objp)
            
            start_x = int(np.min(corners[:,0].T[0]))
            end_x = int(np.max(corners[:,0].T[0]))
            start_y = int(np.min(corners[:,0].T[1]))
            end_y = int(np.max(corners[:,0].T[1]))
            im_gray = cv2.rectangle(im_gray, (start_x, start_y), (end_x, end_y), 0, -1)

    cv2.imshow('im', im_gray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, im_gray.shape[::-1], None, None)

undistort_folders = []
for f in os.listdir(undistort_dir):
    d = os.path.join(undistort_dir, f)
    if os.path.isdir(d): undistort_folders.append(f)
for folder in undistort_folders:
    images = os.listdir(undistort_dir / folder / 'data')
    for image in images:
        im = cv2.imread(str(undistort_dir / folder / 'data' / image))
        im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        h,w = im.shape[:2]
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
        dst = cv2.undistort(im, mtx, dist, None, newcameramtx)
        dst = dst[roi[1]:roi[1]+roi[3],roi[0]:roi[0]+roi[2]]
        (undistorted_dir / folder).mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(undistorted_dir / folder / os.path.split(image)[-1]), dst)

In [11]:
calib_images_path = Path('../data/interim/calib_checkerboards_1by1/image_03')
max_corners = 15
image_to_undistort = Path('../data/raw/final_project/calib/image_03/data/0000000000.png')

image_names = os.listdir(calib_images_path)
objpoints = []
imgpoints = []
vh = []
for v in range(3,max_corners+1):
    for h in range(v,max_corners+1):
        vh.append([v, h])
vh = np.flip(np.asarray(vh))
for image_name in image_names:
    image = cv2.imread(str(calib_images_path / image_name))
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    for v, h in zip([5,7,5], [15,11,7]):
        ret, corners = cv2.findChessboardCorners(image_gray, (v,h))
        if ret: break
    if ret:
        with_corners = cv2.drawChessboardCorners(image_gray, (v,h), corners, ret)
        cv2.imshow('checkerboard', with_corners)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        imgpoints.append(corners)
        objp = np.zeros((v*h,3), np.float32)
        objp[:,:2] = np.mgrid[0:v,0:h].T.reshape(-1,2)
        objpoints.append(objp)
    else:
        print('Checkerboard cannot be found on image '+image_name)
ret, mtx, dist, _, _ = cv2.calibrateCamera(objpoints, imgpoints, image_gray.shape[::-1], None, None)
assert ret, 'Undistortion parameters cannot be calculated.'

image = cv2.imread(str(image_to_undistort))
h, w = image.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
dst = cv2.undistort(image, mtx, dist, None, newcameramtx)
# dst = dst[roi[1]:roi[1]+roi[3],roi[0]:roi[0]+roi[2]]
cv2.imshow('dst', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

Checkerboard cannot be found on image 11.png
Checkerboard cannot be found on image 12.png
Checkerboard cannot be found on image 2.png
Checkerboard cannot be found on image 4.png
Checkerboard cannot be found on image 7.png
Checkerboard cannot be found on image 9.png


In [24]:
image = cv2.imread('../data/interim/calib_checkerboards_1by1/image_02_test/4.png')
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow('im', image_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()
ret, corners = cv2.findChessboardCorners(image_gray, (5,7))
print(ret)

False


In [20]:
calib_image_path = Path('../data/raw/final_project/calib/image_03/data/0000000000.png')
image_to_undistort = Path('../data/raw/final_project/calib/image_03/data/0000000000.png')
max_corners = 15

objpoints = []
imgpoints = []
image = cv2.imread(str(calib_image_path))
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
checkerboards_cnt = 0
for v, h in zip([5,7,5], [15,11,7]):
    ret, corners = cv2.findChessboardCorners(image_gray, (v,h))
    while ret:
        imgpoints.append(corners)
        objp = np.zeros((v*h,3), np.float32)
        objp[:,:2] = np.mgrid[0:v,0:h].T.reshape(-1,2)
        objpoints.append(objp)
        start_x = int(np.min(corners[:,0].T[0]))
        end_x = int(np.max(corners[:,0].T[0]))
        start_y = int(np.min(corners[:,0].T[1]))
        end_y = int(np.max(corners[:,0].T[1]))
        image_gray = cv2.rectangle(image_gray, (start_x, start_y), (end_x, end_y), 0, -1)
        checkerboards_cnt += 1
        ret, corners = cv2.findChessboardCorners(image_gray, (v,h))
print(checkerboards_cnt, 'checkerboards was found on the image.')

ret, mtx, dist, _, _ = cv2.calibrateCamera(objpoints, imgpoints, image_gray.shape[::-1], None, None)
assert ret, 'Undistortion parameters cannot be calculated.'

print(mtx)
print(dist)

import json
data_dict = {'mtx': mtx.tolist(), 'dist': dist.tolist()}
with open('C:/Users/jakab/Desktop/test.json', 'w') as output_file:
    json.dump(data_dict, output_file)
with open('C:/Users/jakab/Desktop/test.json', 'r') as input_file:
    data_dict = json.load(input_file)
mtx = np.asarray(data_dict['mtx'])
dist = np.asarray(data_dict['dist'])

print(mtx)
print(dist)

image = cv2.imread(str(image_to_undistort))
h, w = image.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
dst = cv2.undistort(image, mtx, dist, None, newcameramtx)
dst = dst[roi[1]:roi[1]+roi[3],roi[0]:roi[0]+roi[2]]
#cv2.imshow('dst', dst)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

3 checkerboards was found on the image.
[[978.7638968    0.         707.10302473]
 [  0.         974.5535797  260.05961915]
 [  0.           0.           1.        ]]
[[-0.24549647  0.03799705  0.00259864 -0.00431458 -0.002185  ]]
[[978.7638968    0.         707.10302473]
 [  0.         974.5535797  260.05961915]
 [  0.           0.           1.        ]]
[[-0.24549647  0.03799705  0.00259864 -0.00431458 -0.002185  ]]
